In [6]:
import pandas as pd
import scipy.stats as stats
import helper_functions as hf

In [7]:
# map  rsid to chrom/pos/ref/alt
fn='preprocessed-data/master.geuvadis.annot.parsed.uniq.alleles.reformat.tsv'
df=pd.read_csv(fn,sep='\t',header=None)
df.columns=['chrom','pos','rsid','ref','alt','strand']
df.tail()

,chrom,pos,rsid,ref,alt,strand
33084,4,152646548,rs9998462,T,A,-
33085,7,99700864,rs999886,A,G,+
33086,4,122616863,rs9998908,G,C,-
33087,4,120296210,rs9999028,T,C,+
33088,3,194310359,rs999941,G,A,+


In [8]:
len(df)

33089

In [9]:
Rsid2Cpra={}
Cp2Ra={}

cols=['chrom','pos','ref','alt','rsid']

for chrom,pos,ref,alt,rsid in zip(*[df[ci] for ci in cols]):
    
    if rsid[:2]!='rs': continue
    
    if len(ref)!=1 or len(alt)!=1: continue

    chrom=f'chr{chrom}'
    pos=int(pos)
    
    Rsid2Cpra[rsid]=(chrom,pos,ref,alt)
    
hf.dprint(Rsid2Cpra)

rs10000058 ('chr4', 690366, 'A', 'G')


In [17]:
# Get mpra data
fn='/Users/joe/code/projects/ets/4-ets-opt-in-pub-mpra/8-tewhey-and-gtex/data/tewhey-mpra//tewhey-table-s1.tsv'
df2=pd.read_csv(fn,sep='\t')
df2.head()

,ID,SNP,Direction,Haplotype,C.A.ctrl.mean,C.A.exp.mean,C.A.log2FC,C.A.logP,C.A.logPadj,C.B.ctrl.mean,C.B.exp.mean,C.B.log2FC,C.B.logP,C.B.logPadj,LogSkew.12878,LogSkew.19239,LogSkew.Comb,C.Skew.logP,C.Skew.fdr
0,rs11548103_RC,rs11548103,neg,ref,893.147913,1403.234147,0.637129,20.726159,16.129804,985.132571,1413.418102,0.512199,24.239298,19.642943,-0.157649,-0.070399,-0.124930,1.197350,0.803895
1,rs2016366,rs2016366,pos,ref,316.596386,258.902025,-0.281146,1.799521,0.000000,345.506770,401.928486,0.195118,0.437249,0.000000,0.344054,0.696614,0.476264,NaN,NaN
2,rs2016366_alt,rs2016366,pos,alt,653.148636,605.357051,-0.104744,0.364373,0.000000,627.652912,774.785548,0.287498,1.736553,0.000000,0.390497,0.395150,0.392242,NaN,NaN
3,rs11102212_RC,rs11102212,neg,ref,272.682393,724.187989,1.276380,20.903079,16.306723,270.606119,663.387712,1.183784,18.276201,13.679846,-0.182603,0.057417,-0.092596,0.151356,0.102546
4,rs646867_RC,rs646867,neg,ref,605.412960,595.896429,-0.023234,0.213685,0.000000,978.751908,806.449311,-0.270002,2.243712,0.000000,-0.287121,-0.179513,-0.246768,NaN,NaN


In [19]:
# chrom pos ref alt effect pvalue

c2v={c:[] for c in ['chrom','pos','ref','alt','effect','log-p','p-value']}

n=0
for rsid,effect,logp in zip(*[df2[ci] for ci in ['ID','LogSkew.Comb','C.Skew.fdr']]):
    n+=1

    if len(ref)!=1 or len(alt)!=1: 
        continue
        
    if ':' in rsid:
        # print(rsid in Rsid2Cpra)
        # break
        continue
    
    elif 'MERGED_DEL' in rsid:
        continue
    
    elif '_alt' in rsid: 
        # print(rsid in Rsid2Cpra)
        # break
        continue
        
    elif str(logp)=='nan': 
        continue
    
    elif '_RC' in rsid: 
        rsid=rsid.split('_RC')[0]
        chrom,pos,ref,alt=Rsid2Cpra[rsid]
        
    elif rsid in Rsid2Cpra:
        chrom,pos,ref,alt=Rsid2Cpra[rsid]
        
    else:
        continue


    
    c2v['chrom'].append(chrom)
    c2v['pos'].append(pos)
    c2v['ref'].append(ref)
    c2v['alt'].append(alt)
    c2v['effect'].append(effect)
    c2v['log-p'].append(logp)
    c2v['p-value'].append(10**-logp)
    
outDF=pd.DataFrame(c2v)
outDF.head()
    


,chrom,pos,ref,alt,effect,log-p,p-value
0,chr1,153588340,C,T,-0.124930,0.803895,0.157074
1,chr1,111642031,A,G,-0.092596,0.102546,0.789685
2,chr1,26698747,C,T,-0.120109,0.743469,0.180522
3,chr1,2533552,A,C,-0.365843,1.744575,0.018006
4,chr1,181019202,C,T,-0.334618,0.973007,0.106413


In [22]:
outDF['chrompos']=outDF['chrom']+outDF['pos'].astype(str)
outDF.head()

,chrom,pos,ref,alt,effect,log-p,p-value,chrompos
0,chr1,153588340,C,T,-0.124930,0.803895,0.157074,chr1153588340
1,chr1,111642031,A,G,-0.092596,0.102546,0.789685,chr1111642031
2,chr1,26698747,C,T,-0.120109,0.743469,0.180522,chr126698747
3,chr1,2533552,A,C,-0.365843,1.744575,0.018006,chr12533552
4,chr1,181019202,C,T,-0.334618,0.973007,0.106413,chr1181019202


In [24]:

outDF.to_csv('dataframe-input__fdr__withRC.tsv',sep='\t',index=None)